# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [6]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url)
assert(data.status_code == 200)
data = data.json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2288096935300,
 'week52high': 139.07,
 'week52low': 57.66,
 'week52change': 0.8860576598422052,
 'sharesOutstanding': 17811312775,
 'float': 0,
 'avg10Volume': 134064700,
 'avg30Volume': 111353608,
 'day200MovingAvg': 115.54,
 'day50MovingAvg': 126.96,
 'employees': 0,
 'ttmEPS': 3.3,
 'ttmDividendRate': 0.8375133682227506,
 'dividendYield': 0.0063522126435616826,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-23',
 'nextEarningsDate': '0',
 'peRatio': 39.26658555605837,
 'beta': 1.1697307364367506,
 'maxChangePercent': 49.76399466500338,
 'year5ChangePercent': 4.389492078139312,
 'year2ChangePercent': 2.6514552232229835,
 'year1ChangePercent': 0.9230067984748489,
 'ytdChangePercent': 0.8275425139098025,
 'month6ChangePercent': 0.4934732189288344,
 'month3ChangePercent': 0.1903542272341336,
 'month1ChangePercent': 0.12873733795075867,
 'day30ChangePercent': 0.12475318200210309,
 'day5ChangePercent': 0.0327222480448629}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [7]:
data["year1ChangePercent"]

0.9230067984748489

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [11]:
def chunks_generator(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks_generator(stocks["Ticker"], 100))
symbol_strings = [",".join(symbol_groups[i]) for i in range(len(symbol_groups))]

cols = ["ticker", "stock_price", "one_year_return", "num_of_shares_to_buy"]

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [25]:
df = pd.DataFrame(columns=cols)

for symbol_string in symbol_strings[:1]:
    batch_request_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_request_url)
    assert(data.status_code == 200)
    data = data.json()
    for symbol in symbol_string.split(","):
        df = df.append(pd.Series(
            [
                symbol,
                data[symbol]["price"],
                data[symbol]["stats"]["year1ChangePercent"],
                pd.NA
            ], index=cols
        ), ignore_index=True)

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [27]:
df.sort_values("one_year_return", ascending=False, inplace=True)
df = df.iloc[:50,:].reset_index(drop=True)
df.head()

,ticker,stock_price,one_year_return,num_of_shares_to_buy
0,CARR,39.02,2.214827,<NA>
1,ALB,146.57,1.125326,<NA>
2,AMD,94.17,1.120245,<NA>
3,ALGN,551.82,0.980033,<NA>
4,CDNS,139.90,0.946120,<NA>


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [34]:
portfolio_size = 1_000_000
position_size = portfolio_size / len(df.index)

for i in range(len(df)):
    df.loc[i, "num_of_shares_to_buy"] = math.floor(position_size/df.loc[i, "stock_price"])

---

# Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [99]:
hqm_cols = [
    "ticker", 
    "stock_price", 
    "num_of_shares_to_buy", 
    "one_year_return", 
    "one_year_return_percentile", 
    "six_month_return", 
    "six_month_return_percentile", 
    "three_month_return", 
    "three_month_return_percentile", 
    "one_month_return", 
    "one_month_return_percentile",
    "momentum_score"
    ]

hqm_df = pd.DataFrame(columns=hqm_cols)

for symbol_string in symbol_strings:
    batch_request_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_request_url)
    assert(data.status_code == 200)
    data = data.json()
    for symbol in symbol_string.split(","):
        hqm_df = hqm_df.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]["price"], 
                    pd.NA, 
                    data[symbol]["stats"]["year1ChangePercent"], 
                    pd.NA, 
                    data[symbol]["stats"]["month6ChangePercent"],
                    pd.NA, 
                    data[symbol]["stats"]["month3ChangePercent"],
                    pd.NA, 
                    data[symbol]["stats"]["month1ChangePercent"],
                    pd.NA,
                    pd.NA
                ], 
                index=hqm_cols
            ), 
            ignore_index=True
        )

In [100]:
hqm_df = hqm_df.dropna(subset=["one_year_return"])  # dropping outdated stocks

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [121]:
time_periods = [
    'one_year',
    'six_month',
    'three_month',
    'one_month'
]

for row in hqm_df.index:
    for time_period in time_periods:
        return_col = f"{time_period}_return"
        percentile_col = f"{time_period}_return_percentile"
        hqm_df.loc[row, percentile_col] = stats.percentileofscore(hqm_df[return_col], hqm_df.loc[row, return_col]) / 100

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [122]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = [hqm_df.loc[row, f"{time_period}_return"] for time_period in time_periods]
    hqm_df.loc[row, "momentum_score"] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [123]:
hqm_df = hqm_df.sort_values(by="momentum_score", ascending=False)[:50].reset_index(drop=True)
hqm_df.head()

,ticker,stock_price,num_of_shares_to_buy,one_year_return,one_year_return_percentile,six_month_return,six_month_return_percentile,three_month_return,three_month_return_percentile,one_month_return,one_month_return_percentile,momentum_score
0,CARR,39.62,504,2.22927,1,0.692243,0.74,0.267091,0.28,-0.0102747,0.06,0.794583
1,ALGN,544.14,36,0.974257,0.86,1.11188,0.94,0.672095,0.9,0.206961,0.9,0.741297
2,FCX,25.37,788,0.915477,0.78,1.28441,0.98,0.57083,0.78,0.145747,0.68,0.729116
3,LB,38.00,526,1.09447,0.9,1.61028,1,0.24935,0.24,-0.0508266,0.02,0.725818
4,ALB,153.27,130,1.16606,0.94,0.985447,0.9,0.563045,0.76,0.152482,0.7,0.716759


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [124]:
position_size = 1_000_000/len(hqm_df.index)

for i in hqm_df.index:
    hqm_df.loc[i,"num_of_shares_to_buy"] = math.floor(position_size/hqm_df.loc[i, "stock_price"])

hqm_df.head()

,ticker,stock_price,num_of_shares_to_buy,one_year_return,one_year_return_percentile,six_month_return,six_month_return_percentile,three_month_return,three_month_return_percentile,one_month_return,one_month_return_percentile,momentum_score
0,CARR,39.62,504,2.22927,1,0.692243,0.74,0.267091,0.28,-0.0102747,0.06,0.794583
1,ALGN,544.14,36,0.974257,0.86,1.11188,0.94,0.672095,0.9,0.206961,0.9,0.741297
2,FCX,25.37,788,0.915477,0.78,1.28441,0.98,0.57083,0.78,0.145747,0.68,0.729116
3,LB,38.00,526,1.09447,0.9,1.61028,1,0.24935,0.24,-0.0508266,0.02,0.725818
4,ALB,153.27,130,1.16606,0.94,0.985447,0.9,0.563045,0.76,0.152482,0.7,0.716759


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [128]:
writer = pd.ExcelWriter("momentum_strategy.xlsx", engine="xlsxwriter")
hqm_df.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [129]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

euro_template = writer.book.add_format(
        {
            'num_format':'0.00 €',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [130]:
column_formats = {
    "A": ["ticker", string_template], 
    "B": ["stock_price", euro_template], 
    "C": ["num_of_shares_to_buy", integer_template], 
    "D": ["one_year_return",percent_template], 
    "E": ["one_year_return_percentile", percent_template], 
    "F": ["six_month_return", percent_template], 
    "G": ["six_month_return_percentile", percent_template], 
    "H": ["three_month_return", percent_template], 
    "I": ["three_month_return_percentile", percent_template], 
    "J": ["one_month_return", percent_template], 
    "K": ["one_month_return_percentile", percent_template],
    "L": ["momentum_score", percent_template]
}

for col in column_formats.keys():
    writer.sheets["Momentum Strategy"].set_column(f"{col}:{col}", 25, column_formats[col][1])
    writer.sheets["Momentum Strategy"].write(f"{col}1", column_formats[col][0], column_formats[col][1])

writer.save()